In [1]:
!pip install datasets torch torchvision transformers tensorboard --quiet 
!pip install ipdb --quiet
# colorama

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import sys
import os
from torch.optim import AdamW
import torch
from tqdm.auto import tqdm
import ipdb
import os
import random
import numpy as np
tqdm.pandas()

DRIVE_PATH = "/content/drive/MyDrive/transformers_layers/"
sys.path.append(DRIVE_PATH)

In [4]:
%pdb on

Automatic pdb calling has been turned ON


In [5]:
import shared.utils as utils
import shared.constants as constants
from models.trainer_models import SuperGlueTrainer
from pretrain.trainer_pretrain import PretrainTrainer

utils.set_all_seeds(constants.SEED)

In [6]:
def pretrain_and_train(_task):
    PretrainTrainer(_task).train()
    lm_path = constants.SAVE_LM_PATH + _task
    SuperGlueTrainer(_task, lm_path).train()

In [7]:
SuperGlueTrainer(constants.CB).train()
SuperGlueTrainer(constants.RTE).train()
SuperGlueTrainer(constants.COPA).train()


pretrain_and_train(constants.CB)
pretrain_and_train(constants.RTE)
pretrain_and_train(constants.COPA)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Reusing dataset super_glue (/root/.cache/huggingface/datasets/super_glue/cb/1.0.2/d040c658e2ddef6934fdd97deb45c777b6ff50c524781ea434e7219b56a428a7)


  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


**********************************************************************************************************************************

lm path: None
 cb - eval result: 0.50%
 cb - eval result: 0.50%
**********************************************************************************************************************************



Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Reusing dataset super_glue (/root/.cache/huggingface/datasets/super_glue/rte/1.0.2/d040c658e2ddef6934fdd97deb45c777b6ff50c524781ea434e7219b56a428a7)


...

  0%|          | 0/3 [00:00<?, ?it/s]

n_layers: 14


AttributeError: ignored

> /usr/local/lib/python3.7/dist-packages/transformers/models/roberta/modeling_roberta.py(796)forward()
    794             raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
    795         elif input_ids is not None:
--> 796             input_shape = input_ids.size()
    797         elif inputs_embeds is not None:
    798             input_shape = inputs_embeds.size()[:-1]

ipdb> q



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.7/bdb.py", line 357, in set_quit
    sys.settrace(None)



In [ ]:
# !zip -r all.zip /content/drive/MyDrive/transformers_layers